In [1]:
import sys
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

In [2]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [2]:
user_list = os.listdir('./2018-2019 data/2018-2019 user')
user_file_list = os.listdir(f'./2018-2019 data/2018-2019 user/{user_list[0]}')
user_df = pd.read_csv(f'./2018-2019 data/2018-2019 user/{user_list[0]}/{user_file_list[0]}')
user_df

,ts,action,actionOption,actionSub,actionSubOption,condition,conditionSub1Option,conditionSub2Option,place,emotionPositive,emotionTension,activity
0,1542844920,study,3,0,NaN,3,NaN,NaN,workplace,3,1,3
1,1542844980,study,3,0,NaN,3,NaN,NaN,workplace,3,1,3
2,1542845040,study,3,0,NaN,3,NaN,NaN,workplace,3,1,3
3,1542845100,study,3,0,NaN,3,NaN,NaN,workplace,3,1,3
4,1542845160,study,3,0,NaN,3,NaN,NaN,workplace,3,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
758,1542891720,hobby,3,0,NaN,2,2.0,NaN,other_indoor,5,2,3
759,1542891780,hobby,3,0,NaN,2,2.0,NaN,other_indoor,5,2,3
760,1542891840,hobby,3,0,NaN,2,2.0,NaN,other_indoor,5,2,3
761,1542891900,hobby,3,0,NaN,2,2.0,NaN,other_indoor,5,2,3


# action에 대한 One-Hot-Encoding 진행

In [3]:
feature_list=['action','actionOption','actionSub','actionSubOption','condition','conditionSub1Option','conditionSub2Option', 'place', 'activity']
def make_tmp(user_df,user_list, user_file_list):
    # 한국시간 기준을 위해 timedelta(days=9)를 더함
    tmp = pd.DataFrame({'userId' : [user_list], 'date': [(pd.to_datetime(user_file_list[:-10],unit='s') + timedelta(hours=9)).date().strftime("%Y-%m-%d")]})
    tmp['action_personal_care']=0
    tmp['action_sleep']=0
    tmp['action_work']=0
    tmp['action_study']=0
    tmp['action_household']=0
    tmp['action_recreation_media']=0
    tmp['action_entertainment']=0
    tmp['action_outdoor_act']=0
    tmp['action_hobby']=0
    tmp['action_recreation_etc']=0
    tmp['action_community_interaction']=0
    tmp['action_travel']=0
    tmp['action_meal']=0
    tmp['action_socialising']=0
    
    tmp['actionOption_0']=0
    tmp['actionOption_1']=0
    tmp['actionOption_2']=0
    tmp['actionOption_3']=0
    tmp['actionOption_4']=0
    tmp['actionOption_5']=0
    tmp['actionOption_6']=0
    
    tmp['actionSub_0']=0
    tmp['actionSub_1']=0
    tmp['actionSub_2']=0
    tmp['actionSub_3']=0
    tmp['actionSub_4']=0
    
    tmp['actionSubOption_0']=0
    tmp['actionSubOption_1']=0
    tmp['actionSubOption_2']=0
    tmp['actionSubOption_3']=0
    tmp['actionSubOption_4']=0
    
    tmp['condition_0']=0
    tmp['condition_1']=0
    tmp['condition_2']=0
    tmp['condition_3']=0
    tmp['condition_4']=0
    
    tmp['conditionSub1Option_1']=0
    tmp['conditionSub1Option_2']=0
    tmp['conditionSub1Option_3']=0
    tmp['conditionSub1Option_4']=0
    
    tmp['conditionSub2Option_1']=0
    
    tmp['place_home']=0
    tmp['place_workplace']=0
    tmp['place_restaurant']=0
    tmp['place_outdoor']=0
    tmp['place_other_indoor']=0
    
    tmp['avg_emotionPositive']=user_df['emotionPositive'].mean()
    tmp['avg_emotionTension']=user_df['emotionTension'].mean()

    tmp['activity_0']=0
    tmp['activity_1']=0
    tmp['activity_2']=0
    tmp['activity_3']=0
    tmp['activity_4']=0
    tmp['activity_5']=0
    tmp['activity_6']=0
    tmp['activity_7']=0
    tmp['activity_8']=0
    
    # action에 대해 1분 단위로 하루 누적된 값 적제하기
    for feature in feature_list:
        vc = user_df[feature].value_counts()
        keys = vc.keys()
        for k in keys:
            if type(k)==float:
                tmp[f'{feature}_{int(k)}'] = vc[k]
            else:
                tmp[f'{feature}_{k}'] = vc[k]
   
    return tmp

In [5]:
lifelog_df = pd.DataFrame()
user_list = os.listdir('./2018-2019 data/2018-2019 user')
for ul in user_list:
    user_file_list = os.listdir(f'./2018-2019 data/2018-2019 user/{ul}')
    for ufl in user_file_list:
        user_df = pd.read_csv(f'./2018-2019 data/2018-2019 user/{ul}/{ufl}')
        lifelog_df=pd.concat([lifelog_df,make_tmp(user_df,ul,ufl)])

In [6]:
lifelog_df

,userId,date,action_personal_care,action_sleep,action_work,action_study,action_household,action_recreation_media,action_entertainment,action_outdoor_act,...,activity_0,activity_1,activity_2,activity_3,activity_4,activity_5,activity_6,activity_7,activity_8,action_communitiy_interaction
0,user23,2018-11-22,0,0,0,497,0,0,0,0,...,11,2,43,693,14,0,0,0,0,NaN
0,user23,2018-11-19,0,0,0,289,0,0,0,0,...,17,5,63,472,22,0,0,0,0,NaN
0,user23,2018-11-26,0,0,0,464,0,0,0,0,...,11,2,63,630,27,0,0,0,0,NaN
0,user23,2018-11-23,0,0,0,454,0,0,0,0,...,12,1,42,635,22,0,0,0,0,NaN
0,user23,2018-11-16,40,0,0,478,0,0,0,0,...,26,4,113,546,57,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,user120,2020-01-18,61,0,96,0,0,0,0,0,...,24,0,17,79,96,0,0,0,0,NaN
0,user120,2020-01-20,0,0,0,38,0,0,0,0,...,0,2,37,111,14,0,0,0,0,NaN
0,user120,2020-02-12,45,229,0,0,19,0,0,0,...,10,2,30,555,105,0,0,0,0,NaN
0,user120,2020-01-17,76,0,0,209,22,0,0,0,...,50,1,29,549,78,0,0,0,0,NaN


# 보기 쉽게 userId와 date 기준으로 정렬을 진행

In [7]:
lifelog_df=lifelog_df.sort_values(['userId','date'],axis=0).reset_index(drop=True)
lifelog_df

,userId,date,action_personal_care,action_sleep,action_work,action_study,action_household,action_recreation_media,action_entertainment,action_outdoor_act,...,activity_0,activity_1,activity_2,activity_3,activity_4,activity_5,activity_6,activity_7,activity_8,action_communitiy_interaction
0,user1,2018-11-15,0,0,0,365,0,0,0,0,...,2,1,44,592,12,0,0,0,0,NaN
1,user1,2018-11-16,0,65,0,529,0,0,0,0,...,11,1,60,748,13,0,0,0,0,NaN
2,user1,2018-11-17,19,35,0,118,22,0,0,0,...,1,0,14,594,18,0,0,0,0,NaN
3,user1,2018-11-18,58,134,0,279,21,0,0,0,...,0,1,36,673,19,0,0,0,0,NaN
4,user1,2018-11-19,0,58,121,353,0,0,0,0,...,13,2,88,638,20,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710,user9,2018-11-09,32,0,243,0,24,0,0,5,...,45,5,62,451,47,0,0,0,0,NaN
711,user9,2018-11-10,71,0,0,0,368,0,0,0,...,0,0,49,579,123,0,0,0,0,NaN
712,user9,2018-11-11,104,0,0,40,123,0,0,117,...,18,3,39,628,42,0,0,0,0,32.0
713,user9,2018-11-12,64,0,325,0,62,0,0,0,...,40,5,41,505,36,0,0,0,0,NaN


In [8]:
lifelog_df.describe()

,action_personal_care,action_sleep,action_work,action_study,action_household,action_recreation_media,action_entertainment,action_outdoor_act,action_hobby,action_recreation_etc,...,activity_0,activity_1,activity_2,activity_3,activity_4,activity_5,activity_6,activity_7,activity_8,action_communitiy_interaction
count,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,715.000000,...,715.000000,715.000000,715.000000,715.000000,715.000000,715.0,715.0,715.0,715.0,59.000000
mean,19.036364,11.995804,131.813986,74.201399,21.885315,33.518881,5.376224,5.213986,16.885315,92.201399,...,24.321678,1.618182,50.220979,470.439161,47.734266,0.0,0.0,0.0,0.0,116.932203
std,37.379893,49.807739,167.514213,126.665050,59.529841,105.824286,25.950875,21.403953,54.782849,115.430306,...,37.049063,4.284485,34.505938,168.976504,47.238733,0.0,0.0,0.0,0.0,87.207020
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,0.000000,25.000000,363.000000,18.000000,0.0,0.0,0.0,0.0,50.000000
50%,0.000000,0.000000,32.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51.000000,...,12.000000,1.000000,46.000000,487.000000,36.000000,0.0,0.0,0.0,0.0,107.000000
75%,25.000000,0.000000,248.500000,107.000000,7.500000,0.000000,0.000000,0.000000,0.000000,141.000000,...,30.000000,2.000000,68.000000,583.000000,63.000000,0.0,0.0,0.0,0.0,177.500000
max,285.000000,528.000000,665.000000,622.000000,639.000000,773.000000,300.000000,181.000000,481.000000,674.000000,...,383.000000,70.000000,233.000000,1173.000000,483.000000,0.0,0.0,0.0,0.0,313.000000


In [11]:
lifelog_df.to_csv('./2018-2019 data/2018-2019_lifelog.csv',index = False)